# Fit energies and forces (full cycle)

In [11]:
%load_ext autoreload
%autoreload 2

import os
import sys
import torch
sys.path.append(os.path.expandvars("/home/$USER/repos/fande/"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from ase import io

traj_295 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index=":")
traj_355 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")

traj_295_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
traj_355_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
traj_295_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
traj_355_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")


In [16]:
trajectory_forces = traj_295_2000K[0:5000:10]
trajectory_forces = trajectory_forces[0::10].copy()

trajectory_energy = traj_295[0:5000] + traj_355[0:5000] + traj_295_2000K[0:5000] + traj_355_2000K[0:5000] + traj_295_2000K_forced[0:5000] + traj_355_2000K_forced[0:5000]
trajectory_energy = trajectory_energy[::10].copy()

print(len(trajectory_forces), len(trajectory_energy))

50 3000


In [17]:
from fande.data import FandeDataModule
from fande.utils.find_atomic_groups import find_atomic_groups


soap_params = dict(soap_type="PowerSpectrum",
        interaction_cutoff=3.0,
        max_radial=3,
        max_angular=3,
        gaussian_sigma_constant=0.3,
        gaussian_sigma_type="Constant",
        cutoff_function_type="RadialScaling",
        cutoff_smooth_width=0.1, # 0.1 is way better than 0.5
        cutoff_function_parameters=
                dict(
                        rate=1,
                        scale=3.5,
                        exponent=4
                        ),
        radial_basis="GTO",
        normalize=True, # setting False makes model untrainable
        #   optimization=
        #         dict(
        #                 Spline=dict(
        #                    accuracy=1.0e-05
        #                 )
        #             ),
        compute_gradients=True, # for energies gradients are ignored
        expansion_by_species_method='structure wise'
        )
##FOR NOW USE THE SAME SOAP PARAMETERS FOR ENERGY AND FORCES! (that makes sense if you're modeling the MD)

sample_snapshot = trajectory_forces[0].copy()
fdm = FandeDataModule()
atomic_groups = find_atomic_groups(sample_snapshot)
train_centers_positions = sum(atomic_groups, []) #list(range(len(atoms)))
train_derivatives_positions = sum(atomic_groups, [])#list(range(len(atoms)))
fdm.atomic_groups_sample_snapshot = sample_snapshot.copy()
fdm.atomic_groups = atomic_groups

total_forces_samples_per_group = [3000] * len(atomic_groups)
high_forces_samples_per_group = [0] * len(atomic_groups)

Total number of found groups: 14
Checking if all atoms are covered:  True


In [18]:
dataloader_energy, dataloaders_forces = fdm.dataloaders_from_trajectory(
                                                                trajectory_energy,
                                                                trajectory_forces,
                                                                # energies = None,
                                                                # forces = None,
                                                                atomic_groups = atomic_groups,
                                                                centers_positions = train_centers_positions,
                                                                derivatives_positions = train_derivatives_positions,
                                                                energy_soap_hypers = soap_params,
                                                                forces_soap_hypers = soap_params,
                                                                total_forces_samples_per_group = total_forces_samples_per_group,
                                                                high_force_samples_per_group = high_forces_samples_per_group,
                                                                )

Total length of traj is 3000
Total number of batches 3000
Calculating invariants on trajectory with librascal...


100%|██████████| 3000/3000 [01:21<00:00, 36.80it/s]


invariants for energy fitting calculated


100%|██████████| 50/50 [00:21<00:00,  2.35it/s]


invariants for forces fitting calculated
torch.Size([3000, 540])
torch.Size([3000])
Taking ALL 1800 samples for group 0
Dataloader for group 0 created
Number of samples in dataloader: 1800
Dataloader for group 1 created
Number of samples in dataloader: 3000
Dataloader for group 2 created
Number of samples in dataloader: 3000
Taking ALL 1800 samples for group 3
Dataloader for group 3 created
Number of samples in dataloader: 1800
Taking ALL 1800 samples for group 4
Dataloader for group 4 created
Number of samples in dataloader: 1800
Dataloader for group 5 created
Number of samples in dataloader: 3000
Dataloader for group 6 created
Number of samples in dataloader: 3000
Dataloader for group 7 created
Number of samples in dataloader: 3000
Dataloader for group 8 created
Number of samples in dataloader: 3000
Dataloader for group 9 created
Number of samples in dataloader: 3000
Taking ALL 1800 samples for group 10
Dataloader for group 10 created
Number of samples in dataloader: 1800
Dataloader 

In [115]:
# Making energy model

from fande.models import EnergyModel

hparams = {
        'dtype' : 'float32',
        'device' : 'gpu',
        'energy_model_hparams' : {
                'model_type' : 'variational_inducing_points',#'simple_ard', # 'variational_inducing_points'
                'num_epochs' : 1000,
                'learning_rate' : 0.01,
        }
        }
       
Energy_model = EnergyModel(
        dataloader_energy,
        hparams=hparams)

Energy_model.fit()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name       | Type                    | Params
-------------------------------------------------------
0 | likelihood | GaussianLikelihood      | 1     
1 | model      | SVGPModelEnergies       | 64.6 K
2 | mll        | PredictiveLogLikelihood | 64.6 K
-------------------------------------------------------
64.6 K    Trainable params
0         Non-trainable params
64.6 K    Total params
0.259     Total estimated model params size (MB)


Training with inducing points:  torch.Size([100, 540])
Training energy model


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1000` reached.


In [ ]:
# %%time
# # Fitting forces

# from fande.models import ModelForces, GroupModelForces


# n_steps_list = [200] * len(atomic_groups)
# lr_list = [0.1] * len(atomic_groups)

# models_hparams = []
# for i in range(len(atomic_groups)):
#         model_hparams = {
#         'atomic_group' : atomic_groups[i],
#         'dtype' : hparams['dtype'],
#         'device' : hparams['device'],
#         'num_epochs' : n_steps_list[i],
#         'learning_rate' : lr_list[i],
#         'soap_dim' : dataloaders_forces[i].dataset[0][0].shape[-1],
#         'soap_params' : soap_params,
#         }
#         models_hparams.append(model_hparams)

# hparams['per_model_hparams'] = models_hparams # access per_model_hparams by model.model_id
# gpu_id = 0


# models_forces = []
# for i in range(len(atomic_groups)):
#         model = ModelForces(
#         train_x = dataloaders_forces[i].dataset[:][0],
#         train_y = dataloaders_forces[i].dataset[:][1],
#         atomic_group = atomic_groups[i],
#         hparams = hparams,
#         id=i)
#         models_forces.append(model)
        
# AG_force_model = GroupModelForces(
#         models= models_forces,
#         train_data_loaders = dataloaders_forces,
#         hparams=hparams, 
#         gpu_id=gpu_id)

# AG_force_model.fit()

In [120]:
from fande.predict import FandePredictor
from fande.ase import FandeCalc

# Energy_model = None
AG_force_model = None
predictor = FandePredictor(
        fdm,
        AG_force_model,
        Energy_model,
        hparams,
        soap_params
        )

fande_calc = FandeCalc(predictor)
# fande_calc.save_predictor("/data1/simulations/ML_models/for_SCML/fande_predictor_3000.pth")

In [38]:

from fande.predict import FandePredictor
from fande.ase import FandeCalc
# load the predictor:
predictor_loaded = torch.load("/data1/simulations/ML_models/for_SCML/fande_predictor.pth")
fande_calc_loaded = FandeCalc(predictor_loaded)
device = torch.device('cpu')
fande_calc_loaded.predictor.move_models_to_device(device)

In [88]:
# device = torch.device('cuda:0') # always specify the gpu id!
device = torch.device('cpu')
fande_calc.predictor.move_models_to_device(device)

In [121]:
from ase import io
from tqdm import tqdm
test_traj = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="1235:1240")
test_traj = test_traj.copy()

real_energies = [s.get_potential_energy() for s in test_traj]
predicted_energies = []
for i in tqdm(range(len(test_traj))):
        test_traj[i].calc = fande_calc
        # predicted_energies.append( test_traj[i].get_potential_energy() )
        # print(test_traj[i].get_potential_energy() )
        # test_traj[i].get_forces()
        print(test_traj[i].get_potential_energy() )
        # test_traj[i].get_forces()

 20%|██        | 1/5 [00:00<00:01,  2.05it/s]

Time for invariants (call from forces):  460.8292579650879
removing cache for energy model (THIS IS EXPENSIVE) 
Cache moved to the proper device.
Energy model summary: 
Time invariants:  0.12350082397460938
Time prediction:  6.623983383178711
Time moving on device:  0.8473396301269531
Time total:  23.772478103637695
[-3082.5542]


 40%|████      | 2/5 [00:00<00:01,  2.14it/s]

Time for invariants (call from forces):  444.5202350616455
removing cache for energy model (THIS IS EXPENSIVE) 
Cache moved to the proper device.
Energy model summary: 
Time invariants:  0.10824203491210938
Time prediction:  4.958152770996094
Time moving on device:  0.7417201995849609
Time total:  7.117509841918945
[-3082.7007]


 60%|██████    | 3/5 [00:01<00:00,  2.25it/s]

Time for invariants (call from forces):  406.62288665771484
removing cache for energy model (THIS IS EXPENSIVE) 
Cache moved to the proper device.
Energy model summary: 
Time invariants:  0.10633468627929688
Time prediction:  4.934787750244141
Time moving on device:  0.7455348968505859
Time total:  7.107019424438477
[-3082.8933]


 80%|████████  | 4/5 [00:01<00:00,  2.30it/s]

Time for invariants (call from forces):  408.42509269714355
removing cache for energy model (THIS IS EXPENSIVE) 
Cache moved to the proper device.
Energy model summary: 
Time invariants:  0.10800361633300781
Time prediction:  4.927158355712891
Time moving on device:  0.7460117340087891
Time total:  7.096290588378906
[-3083.0618]


100%|██████████| 5/5 [00:02<00:00,  2.27it/s]

Time for invariants (call from forces):  409.27958488464355
removing cache for energy model (THIS IS EXPENSIVE) 
Cache moved to the proper device.
Energy model summary: 
Time invariants:  0.10704994201660156
Time prediction:  4.898309707641602
Time moving on device:  0.7574558258056641
Time total:  7.070064544677734
[-3083.136]


In [92]:
%%time
# test_traj[0].get_forces()
for i in tqdm(range(len(test_traj))):
        test_traj[i].get_forces()
# test_traj[1].get_potential_energy()

Time for invariants (call from forces):  425.555944442749
Energy model summary: 
Time invariants:  0.10633468627929688
Time prediction:  3.1805038452148438
Time moving on device:  0.4601478576660156
Time total:  4.599332809448242


Time for invariants (call from forces):  408.1246852874756
Energy model summary: 
Time invariants:  0.10585784912109375
Time prediction:  3.0345916748046875
Time moving on device:  0.4558563232421875
Time total:  4.397392272949219


Time for invariants (call from forces):  406.5885543823242
Energy model summary: 
Time invariants:  0.1068115234375
Time prediction:  3.0775070190429688
Time moving on device:  0.4553794860839844
Time total:  4.43720817565918


100%|██████████| 5/5 [00:01<00:00,  2.97it/s]

Time for invariants (call from forces):  407.9163074493408
Energy model summary: 
Time invariants:  0.11110305786132812
Time prediction:  3.017902374267578
Time moving on device:  0.4487037658691406
Time total:  4.424810409545898
CPU times: user 6.01 s, sys: 329 ms, total: 6.34 s
Wall time: 1.69 s


In [ ]:
import matplotlib.pyplot as plt

plt.plot(real_energies, label="real")
plt.plot(predicted_energies, label="predicted")
plt.legend()
plt.show()

In [14]:
atoms = trajectory_energy[51].copy()

atoms.set_calculator(fande_calc)

print(atoms.get_potential_energy())
print(atoms.get_forces())

Atomic group force model is not defined. Cannot predict forces. Returning zeros.
[-3080.6099]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


## Testing area

In [135]:
from ase import io
test_traj = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="-100:")
# test_traj = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index="-100:")


# atoms.set_calculator(fande_calc_loaded)

In [136]:
%%capture c
energies_true = []
energies_pred = []

for i in tqdm(range(len(test_traj))):
        atoms = test_traj[i].copy()
        atoms.calc = fande_calc
        energies_pred.append(atoms.get_potential_energy())
        energies_true.append(test_traj[i].get_potential_energy())


In [ ]:
import matplotlib.pyplot as plt
plt.plot(energies_true, label="true")
plt.plot(energies_pred, label="pred")
plt.legend()
plt.show()